
# Datos geográficos



In [2]:
import urllib
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import fiona

## Normalización departamentos


In [3]:
import requests as rqs

In [21]:
df= pd.read_csv("snic-departamentos.csv", sep=";", encoding="utf8",
               converters={"provincia_id": lambda x: str(x).zfill(2), "dpto_nombre": str, "dpto_id": str})
df.head(2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29594 entries, 0 to 29593
Data columns (total 9 columns):
anio                             29580 non-null float64
provincia_id                     29594 non-null object
provincia_nombre                 29580 non-null object
dpto_id                          29594 non-null object
dpto_nombre                      29594 non-null object
codigo_delito_agrupado_nombre    29580 non-null object
cantidad_hechos                  12325 non-null float64
cantidad_victimas                17255 non-null float64
Unnamed: 8                       0 non-null float64
dtypes: float64(4), object(5)
memory usage: 2.0+ MB


In [5]:
df= pd.DataFrame(df)

In [6]:
df=df[~df.provincia_id.isnull()]
df["provincia_id"]=df[~df.provincia_id.isnull()]["provincia_id"].astype(int)

In [7]:
len(df)

29594

In [8]:
departamentos_para_georef = [
    {"departamento": row[1]["dpto_nombre"], 
     "provincia_id": str(row[1]["provincia_id"]).zfill(2)}
    for row in df[["dpto_nombre", "provincia_id"]].drop_duplicates().iterrows()
]
departamentos_para_georef 

[{'departamento': '25 de Mayo', 'provincia_id': '06'},
 {'departamento': '9 de Julio', 'provincia_id': '06'},
 {'departamento': 'Adolfo Alsina', 'provincia_id': '06'},
 {'departamento': 'Adolfo Gonzales Chaves', 'provincia_id': '06'},
 {'departamento': 'Alberti', 'provincia_id': '06'},
 {'departamento': 'Almirante Brown', 'provincia_id': '06'},
 {'departamento': 'Arrecifes', 'provincia_id': '06'},
 {'departamento': 'Avellaneda', 'provincia_id': '06'},
 {'departamento': 'Ayacucho', 'provincia_id': '06'},
 {'departamento': 'Azul', 'provincia_id': '06'},
 {'departamento': 'Bahia Blanca', 'provincia_id': '06'},
 {'departamento': 'Balcarce', 'provincia_id': '06'},
 {'departamento': 'Baradero', 'provincia_id': '06'},
 {'departamento': 'Benito Juarez', 'provincia_id': '06'},
 {'departamento': 'Berazategui', 'provincia_id': '06'},
 {'departamento': 'Berisso', 'provincia_id': '06'},
 {'departamento': 'Bolivar', 'provincia_id': '06'},
 {'departamento': 'Bragado', 'provincia_id': '06'},
 {'depart

In [9]:
len(departamentos_para_georef)

494

In [10]:
#Elimino la última fila por estar ésta vacía.
departamentos_para_georef[:493]

[{'departamento': '25 de Mayo', 'provincia_id': '06'},
 {'departamento': '9 de Julio', 'provincia_id': '06'},
 {'departamento': 'Adolfo Alsina', 'provincia_id': '06'},
 {'departamento': 'Adolfo Gonzales Chaves', 'provincia_id': '06'},
 {'departamento': 'Alberti', 'provincia_id': '06'},
 {'departamento': 'Almirante Brown', 'provincia_id': '06'},
 {'departamento': 'Arrecifes', 'provincia_id': '06'},
 {'departamento': 'Avellaneda', 'provincia_id': '06'},
 {'departamento': 'Ayacucho', 'provincia_id': '06'},
 {'departamento': 'Azul', 'provincia_id': '06'},
 {'departamento': 'Bahia Blanca', 'provincia_id': '06'},
 {'departamento': 'Balcarce', 'provincia_id': '06'},
 {'departamento': 'Baradero', 'provincia_id': '06'},
 {'departamento': 'Benito Juarez', 'provincia_id': '06'},
 {'departamento': 'Berazategui', 'provincia_id': '06'},
 {'departamento': 'Berisso', 'provincia_id': '06'},
 {'departamento': 'Bolivar', 'provincia_id': '06'},
 {'departamento': 'Bragado', 'provincia_id': '06'},
 {'depart

In [11]:
def get_similar_bulk(departamentos):
    """Normaliza una lista de nombres de alguna de las entidades geográficas."""
    
    # realiza consulta a la API
    data = {
        "departamentos": [
            {
             "nombre": departamento["departamento"],
             "provincia": departamento["provincia_id"],
             "max": 1,
             "aplanar": True
            } 
            for departamento in departamentos
    ]}
    
    #
    url_endpoint = "https://apis.datos.gob.ar/georef/api/departamentos"
    try:
        results = requests.post(
            url_endpoint, json=data, headers={"Content-Type": "application/json"}
        )
        results_json = results.json()
    except Exception as e:
        print(e)
        print("No se pudieron geolocalizar {} departamentos".format(len(departamentos)))
        return []

    departamentos_normalizados = []
    for resultado, departamento in zip(results_json["resultados"], departamentos):
        if len(resultado["departamentos"]) > 0:
            departamento_normalizado = resultado["departamentos"][0]
            departamento_normalizado["departamento_original"] = departamento["departamento"]
            departamentos_normalizados.append(departamento_normalizado)
    
    return departamentos_normalizados
    

In [12]:
#%pdb

In [13]:
departamentos_normalizados= get_similar_bulk(departamentos_para_georef[:493])

In [14]:
departamentos_normalizados[0]

{'centroide_lat': -35.5265113228112,
 'centroide_lon': -60.2301576569223,
 'id': '06854',
 'nombre': '25 de Mayo',
 'provincia_id': '06',
 'provincia_nombre': 'Buenos Aires',
 'departamento_original': '25 de Mayo'}

In [15]:
df_deptos_normalizados = pd.DataFrame(departamentos_normalizados)

In [16]:
len(df_deptos_normalizados), len(df_deptos_normalizados.departamento_original.unique())

(483, 405)

In [17]:
df_deptos_normalizados

,centroide_lat,centroide_lon,departamento_original,id,nombre,provincia_id,provincia_nombre
0,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,06,Buenos Aires
1,-35.481191,-60.975116,9 de Julio,06588,9 de Julio,06,Buenos Aires
2,-37.196336,-63.056007,Adolfo Alsina,06007,Adolfo Alsina,06,Buenos Aires
3,-37.964225,-60.249078,Adolfo Gonzales Chaves,06014,Adolfo Gonzales Chaves,06,Buenos Aires
4,-35.036382,-60.282869,Alberti,06021,Alberti,06,Buenos Aires
5,-34.835570,-58.367398,Almirante Brown,06028,Almirante Brown,06,Buenos Aires
6,-34.011315,-60.062763,Arrecifes,06077,Arrecifes,06,Buenos Aires
7,-34.678206,-58.341117,Avellaneda,06035,Avellaneda,06,Buenos Aires
8,-37.034782,-58.442581,Ayacucho,06042,Ayacucho,06,Buenos Aires
9,-36.785772,-59.696652,Azul,06049,Azul,06,Buenos Aires


In [32]:
departamentos_normalizados_vf= df.merge(df_deptos_normalizados, "left", 
         left_on=["provincia_id", "dpto_nombre"], 
         right_on=["provincia_id", "departamento_original"]
        )
len(departamentos_normalizados_vf), len(df)

(29594, 29594)

In [37]:
departamentos_normalizados_vf.columns= ['anio', 'provincia_id', 'provincia_nombre_x', 'dpto_id', 'dpto_nombre',
       'codigo_delito_agrupado_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'Unnamed: 8', 'centroide_lat', 'centroide_lon', 'departamento_original',
       'id', 'nombre', 'provincia_nombre_y']

In [38]:
departamentos_normalizados_vf=

,anio,provincia_id,provincia_nombre_x,dpto_id,dpto_nombre,codigo_delito_agrupado_nombre,cantidad_hechos,cantidad_victimas,Unnamed: 8,centroide_lat,centroide_lon,departamento_original,id,nombre,provincia_nombre_y
0,2014.0,06,Buenos Aires,,25 de Mayo,Total de hechos delictivos,261.0,NaN,NaN,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,Buenos Aires
1,2014.0,06,Buenos Aires,,25 de Mayo,Total de vmctimas de delitos contra las personas,NaN,3.0,NaN,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,Buenos Aires
2,2014.0,06,Buenos Aires,,25 de Mayo,Total de vmctimas de homicidios dolosos,NaN,2.0,NaN,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,Buenos Aires
3,2014.0,06,Buenos Aires,,25 de Mayo,Total de vmctimas de lesiones dolosas,NaN,0.0,NaN,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,Buenos Aires
4,2014.0,06,Buenos Aires,,25 de Mayo,Total de vmctimas de muertes en siniestros viales,NaN,1.0,NaN,-35.526511,-60.230158,25 de Mayo,06854,25 de Mayo,Buenos Aires


In [31]:
departamentos_normalizados_vf.to_excel("departamentos_norm.xlsx", encoding="utf8")